# Algorithmic Question

The idea we had to solve this problem is to model it through a directed graph whit nine nodes and edges connecting those nodes expressing with their weights the direction they're pointing. The directions are 'U', 'D', 'R', 'L', as the input examples suggested. 

Then we defined a first naive approach given a strong assumption that simplifies the problem. 


<div align='center'> EACH DIRECTION IN THE INPUT STRING CAN ONLY BE RUN ACROSS ONE TIME</div>

This is a very approximative pseudo-code for our idea:

```
PathChecker(INPUT: G:graph, N: node, Directions: str, OUTPUT: Boolean):
    if len(Directions) == 0:
        return(any in G(node).weight == Directions[0])
    else:
        if any in G(node).weights == Directions[0]:
            edge <- edge in G.edges: edge.weight == Directions[0]
            N <- edge[1]
            return(PathChecker(G, N, Directions[1:]))
        else:
            return False
```

We implemented a class to deal with directed graphs from scratch:
- `__init__` is the class builder that takes in input a list of nodes and a dictionary for the edges and builds a `adDict` for the adjacency to have a self-representation. It also initializes a variable called `visited` that is useful in subsequent methods;
- `getNeighbours` extract the neighbours of a node through the adjacency matrix;
- `getConnections` returns as a list the exit-star of a node. In other words it outputs the edges that have the tail in the node. 
- `findDirection` check if from a node there's an edge pointing in a certain direction, and eventually outputs that edge.
- `visitedInitialization` initialize the `self.visited` to a list containing the source of a walk. It is useful to take track of the visited node, since one of the constraints is that the walk on the phone-polyline must not pass over an already touched node. 
- `checkPath` recursively leverages `findDirection` over a string of directions to check if exists a succession of directions path from a node.
- `countPath` counts how many paths are looping through all the nodes.

```

In [1]:
class Graph:
    def __init__(self, nodes:list, edges:dict):
        self.nodes = nodes
        self.edges = edges
        self.visited = None
        self.adDict = {node: [] for node in self.nodes}

        for (src, dst) in self.edges.keys():
            self.adDict[src].append(dst)
            self.adDict[dst].append(src)
    
    def getNeighbours(self, node):
        return self.adDict[node]
    
    def getConnections(self, node):
        return [edge for edge in self.edges.keys() if edge[0] == node]

    def findDirection(self, node, direction):
        for edge in self.getConnections(node):
            if self.edges[edge] == direction:
                return edge 
        return False
    
    def visitedInitialization(self,node):
        self.visited = [node]
    
    def checkPath(self, node, directions:str):
        if len(directions) == 1:
            if self.findDirection(node,directions):
                edge = self.findDirection(node, directions)
                pointed = edge[1]
                if pointed in self.visited:
                    return False
                else:
                    return True
            else:
                return False
        else:
            if self.findDirection(node, directions[0]):
                edge = self.findDirection(node, directions[0])
                pointed = edge[1]
                if pointed in self.visited:
                    return False
                else:
                    self.visited.append(pointed)
                    return self.checkPath(edge[1], directions[1:])
            else:
                return False

    def countPath(self, directions):
        count = 0
        for node in self.nodes:
            self.visitedInitialization(node)
            count += int(self.checkPath(node, directions))
        return count

This is the instance of the graph that modelize our problem: as we can see we have redundancies in the edge list that could have been solved with an undirected graph, but this instanciation is more expressive for our problem.

In [2]:
phone_polyline = Graph([1,2,3,4,5,6,7,8,9],{(1,2):'R',(2,1):'L',
                                            (4,1):'U',(1,4):'D',
                                            (2,3):'R',(3,2):'L',
                                            (2,5):'D',(5,2):'U',
                                            (3,6):'D',(6,3):'U',
                                            (4,7):'D',(7,4):'U',
                                            (4,5):'R',(5,4):'L',
                                            (5,6):'R',(6,5):'L',
                                            (5,8):'D',(8,5):'U',
                                            (6,9):'D',(9,6):'U',
                                            (7,8):'R',(8,7):'L',
                                            (8,9):'R',(9,8):'L'})

Now we can modify our initial assumption:

<div align='center'> EACH DIRECTION IN THE INPUT STRING CAN BE RUN ACROSS ONCE OR TWICE</div>

That leads to another recursive question: we need to define all the possible paths that come down from a string of directions. In our `Solution` class we implemented a `directionsRecombiner` method that does so. Let's see an example:

In [16]:
sol = Solution(phone_polyline)
sol.directionsRecombiner('DRU')

['DRU', 'DRUU', 'DRRU', 'DRRUU', 'DDRU', 'DDRUU', 'DDRRU', 'DDRRUU']

As a binary splitting problem from a string of length $L$ we get $2^L$ possible paths. We are going to count how many of them are actually admittable on every node of the graph as the source to get the solution to the problem.

In [9]:
class Solution:
    def __init__(self, graph: Graph):
        self.graph = graph
    
    def directionsRecombiner(self, directions):
        if len(directions) == 1:
            return [directions, 2*directions]
        else:
            list1 = [directions[0]+ricomb for ricomb in self.directionsRecombiner(directions[1:])]
            list2 = [2*directions[0]+ricomb for ricomb in self.directionsRecombiner(directions[1:])]
            return list1 + list2

    def finalCounter(self, directions):
        final_count = 0
        for direction in self.directionsRecombiner(directions):
            final_count += self.graph.countPath(direction)
        return final_count

In [10]:
sol = Solution(phone_polyline)
print(sol.finalCounter('DRU'))
print(sol.finalCounter('R'))
print(sol.finalCounter('LDRDLUL'))

15
9
0
